In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
     |████████████████████████████████| 895 kB 44.5 MB/s 
     |████████████████████████████████| 61 kB 506 kB/s 
     |████████████████████████████████| 3.3 MB 52.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install torch

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1NbH-3LL4bqKmunw388J037IJvXlr_kDW' -O 'train_data_lag_5.csv'
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1hz_zmgWPN0mSczWV2m4aq9p4NRNdp4rD' -O 'val_data_lag_5.csv'
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1v63Zbg5PizgCqhG9FopdO-i1iWuXFo8D' -O 'test_data_lag_5.csv'

--2021-12-21 02:45:10--  https://drive.google.com/uc?export=download&id=1NbH-3LL4bqKmunw388J037IJvXlr_kDW
Resolving drive.google.com (drive.google.com)... 74.125.31.138, 74.125.31.113, 74.125.31.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.31.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/s6039m6338ldg06p9mm1pdmn63jqvd8t/1640054700000/14724390508833084599/*/1NbH-3LL4bqKmunw388J037IJvXlr_kDW?e=download [following]
--2021-12-21 02:45:13--  https://doc-0g-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/s6039m6338ldg06p9mm1pdmn63jqvd8t/1640054700000/14724390508833084599/*/1NbH-3LL4bqKmunw388J037IJvXlr_kDW?e=download
Resolving doc-0g-18-docs.googleusercontent.com (doc-0g-18-docs.googleusercontent.com)... 173.194.218.132, 2607:f8b0:400c:c14::84
Connecting to doc-0g-18-docs.googleusercontent.com (doc-0g-18

In [ ]:
!ls

sample_data		   train_data_lag_5.csv        val_data_lag_5_paper.csv
test_data_lag_5.csv	   train_data_lag_5_paper.csv
test_data_lag_5_paper.csv  val_data_lag_5.csv


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

if torch.cuda.is_available():

    device = torch.device("cuda")
    print('GPU Name:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
import datetime
%matplotlib inline
sns.set(color_codes=True)

In [ ]:
train = pd.read_csv('train_data_lag_5.csv',lineterminator='\n')
train['tweet'] = train['tweet'].str.strip()

val = pd.read_csv('val_data_lag_5.csv',lineterminator='\n')
val['tweet'] = val['tweet'].str.strip()

In [ ]:
train['market_binary'] = train['market_binary'].apply(int)
val['market_binary'] = val['market_binary'].apply(int)

In [ ]:
train_labels = train.market_binary.values
train_sentences = train.tweet.values

In [ ]:
val_labels = val.market_binary.values
val_sentences = val.tweet.values

In [ ]:
epochs = 5
max_len = 512

In [ ]:
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
print(' Original: ', train_sentences[0])
print('Tokenized: ', tokenizer.tokenize(train_sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_sentences[0])))

 Original:  $ aapl - wall st . kicks off new year on lower note -> URL stock stocks stockaction rt AT_USER here's how apple could be making a huge push into healthcare --> URL $ aapl rt AT_USER our top 3 trade ideas for 2014 and two $ 1,000 futures wins already in the bank $ gld $ uso $ aapl - - URL AT_USER $ aapl having breathing difficulties emylers : cheeky's board ! : $ aapl up to date company activities and other impo URL $ aapl blackberry and singer alicia keys to part ways URL $ aapl - chinas xiaomi says to more than double smartphone sales in 2014 -> URL stock stocks stockaction rt AT_USER our top 3 trade ideas for 2014 and two $ 1,000 futures wins already in the bank $ gld $ uso $ aapl - - URL rt AT_USER our top 3 trade ideas for 2014 and two $ 1,000 futures wins already in the bank $ gld $ uso $ aapl - - URL what are your top picks for q1 2014 ? here are mine : $ aapl $ cree $ fb $ cldx $ sne rt AT_USER our top 3 trade ideas for 2014 and two $ 1,000 futures wins already in th

In [ ]:
train_input_ids = []
train_attention_masks = []

for sent in train_sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = max_len,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )

    train_input_ids.append(encoded_dict['input_ids'])
    train_attention_masks.append(encoded_dict['attention_mask'])

train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

print('Original: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  $ aapl - wall st . kicks off new year on lower note -> URL stock stocks stockaction rt AT_USER here's how apple could be making a huge push into healthcare --> URL $ aapl rt AT_USER our top 3 trade ideas for 2014 and two $ 1,000 futures wins already in the bank $ gld $ uso $ aapl - - URL AT_USER $ aapl having breathing difficulties emylers : cheeky's board ! : $ aapl up to date company activities and other impo URL $ aapl blackberry and singer alicia keys to part ways URL $ aapl - chinas xiaomi says to more than double smartphone sales in 2014 -> URL stock stocks stockaction rt AT_USER our top 3 trade ideas for 2014 and two $ 1,000 futures wins already in the bank $ gld $ uso $ aapl - - URL rt AT_USER our top 3 trade ideas for 2014 and two $ 1,000 futures wins already in the bank $ gld $ uso $ aapl - - URL what are your top picks for q1 2014 ? here are mine : $ aapl $ cree $ fb $ cldx $ sne rt AT_USER our top 3 trade ideas for 2014 and two $ 1,000 futures wins already in the

In [ ]:
val_input_ids = []
val_attention_masks = []

for sent in val_sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = max_len,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    
    val_input_ids.append(encoded_dict['input_ids'])
    val_attention_masks.append(encoded_dict['attention_mask'])

val_input_ids = torch.cat(val_input_ids, dim=0)
val_attention_masks = torch.cat(val_attention_masks, dim=0)
val_labels = torch.tensor(val_labels)

print('Original: ', val_sentences[0])
print('Token IDs:', val_input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  $ goog has no such power over $ aapl . the next few months are going to be fun to watch . get your popcorn ready . ashr db x-trackers harvest csi 300 china a-shares fund e . p . s . URL $ ashr $ ubs $ aapl $ lmt ashr tradeideas … himx himax technologies inc . 52wk low URL $ himx $ aapl $ blue $ bidu himx invest pennystocks AT_USER hi albertine , thks for fl see my last price target for $ aapl $ twtr $ tsla $ goog on URL AT_USER hi patricia , thks for fl see my last price target for $ aapl $ twtr $ tsla $ goog on URL $ googl google URL greferendum $ dyax $ hban $ gt $ dhi $ cisco $ bk . b $ aapl $ nflx $ woof $ pzza $ swks $ fb $ ctas $ oslh $ amzn toyota picks this little-known company over apple , google $ aapl URL $ googl google URL $ dyax $ hban $ gt $ dhi $ cisco $ bk . b $ aapl $ nflx $ woof $ pzza $ swks $ fb $ ctas $ googl $ zqk $ fv AT_USER hi the , thks for fl see my last price target for $ aapl $ twtr $ tsla $ goog on URL plenty of growth for iphone , helped by and

In [ ]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

In [ ]:
from torch.utils.data import DataLoader,RandomSampler, SequentialSampler

batch_size = 16
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size,
            shuffle = False
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size,
            shuffle = False
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import logging
logging.set_verbosity_warning()
logging.set_verbosity_error()

reinit_layers = 3
_model_type = 'bert'
_pretrained_model = 'bert-base-uncased'
config = BertConfig.from_pretrained(_pretrained_model)
model = BertForSequenceClassification.from_pretrained(_pretrained_model, num_labels = 2,
    output_attentions = False,
    output_hidden_states = False)

if reinit_layers > 0:
    print(f'Reinitializing Last {reinit_layers} Layers ...')
    encoder_temp = getattr(model, _model_type)
    for layer in encoder_temp.encoder.layer[-reinit_layers:]:
        for module in layer.modules():
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=config.initializer_range)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(mean=0.0, std=config.initializer_range)
                if module.padding_idx is not None:
                    module.weight.data[module.padding_idx].zero_()
            elif isinstance(module, nn.LayerNorm):
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)
    print('Done.!')
model.cuda()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Reinitializing Last 3 Layers ...
Done.!


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def bert_base_adamw_grouped_llrd(model):
        
    opt_parameters = []
    named_parameters = list(model.named_parameters()) 
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    set_2 = ["layer.4", "layer.5", "layer.6", "layer.7"]
    set_3 = ["layer.8", "layer.9", "layer.10", "layer.11"]
    init_lr = 1e-8
    
    for i, (name, params) in enumerate(named_parameters):  
        
        weight_decay = 0.0 if any(p in name for p in no_decay) else 0.01
        lr = init_lr       
        lr = init_lr * 1.75 if any(p in name for p in set_2) else lr
        lr = init_lr * 35 if any(p in name for p in set_3) else lr
        opt_parameters.append({"params": params,
                                "weight_decay": weight_decay,
                                "lr": lr})
    return AdamW(opt_parameters, lr=init_lr)

In [ ]:
optimizer = bert_base_adamw_grouped_llrd(model)

In [ ]:
from transformers import get_linear_schedule_with_warmup

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 50,
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
from sklearn.metrics import matthews_corrcoef

In [ ]:
import random
import numpy as np

seed_val = 26

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()

    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):

        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()        
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)
        loss = result.loss
        logits = result.logits
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.3f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    total_eval_mcc = 0
    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():        

            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        loss = result.loss
        logits = result.logits
            
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        total_eval_mcc += matthews_corrcoef(np.argmax(logits, axis=1).flatten(), label_ids.flatten())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_mcc = total_eval_mcc / len(validation_dataloader)
    print("  Accuracy: {0:.3f}".format(avg_val_accuracy))
    print("  Validation MCC: {0:.3f}".format(avg_val_mcc))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...
  Batch   500  of    937.    Elapsed: 0:07:12.

  Average training loss: 0.699
  Training epoch took: 0:13:30

Running Validation...
  Accuracy: 0.488
  Validation MCC: -0.012
  Validation Loss: 0.695
  Validation took: 0:00:36

======== Epoch 2 / 5 ========
Training...
  Batch   500  of    937.    Elapsed: 0:07:12.

  Average training loss: 0.699
  Training epoch took: 0:13:30

Running Validation...
  Accuracy: 0.491
  Validation MCC: -0.004
  Validation Loss: 0.696
  Validation took: 0:00:36

======== Epoch 3 / 5 ========
Training...
  Batch   500  of    937.    Elapsed: 0:07:13.

  Average training loss: 0.697
  Training epoch took: 0:13:30

Running Validation...
  Accuracy: 0.482
  Validation MCC: -0.002
  Validation Loss: 0.697
  Validation took: 0:00:36

======== Epoch 4 / 5 ========
Training...
  Batch   500  of    937.    Elapsed: 0:07:13.

  Average training loss: 0.698
  Training epoch took: 0:13:30

Running Validation...
  Accuracy:

In [ ]:
pd.set_option('precision', 2)
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.7,0.7,0.49,0:13:30,0:00:36
2,0.7,0.7,0.49,0:13:30,0:00:36
3,0.7,0.7,0.48,0:13:30,0:00:36
4,0.7,0.7,0.48,0:13:30,0:00:36
5,0.7,0.7,0.48,0:13:31,0:00:36


In [ ]:
df = pd.read_csv("test_data_lag_5.csv",lineterminator='\n')
df['tweet'] = df['tweet'].str.strip()
df['market_binary'] = df['market_binary'].apply(int)
df = df.dropna()

print('Number of test sentences: {:,}\n'.format(df.shape[0]))
sentences = df.tweet.values
labels = df.market_binary.values
input_ids = []
attention_masks = []

for sent in sentences:

    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = max_len,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
batch_size = 16
prediction_data = TensorDataset(input_ids, attention_masks,labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 2,737



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

model.eval()
predictions ,predictions_probs, true_labels = [], [], []
for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask,b_labels = batch

  with torch.no_grad():
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits
  probabilities = F.softmax(logits, dim=1)
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  probabilities = probabilities.detach().cpu().numpy()
  predictions.append(logits)
  true_labels.append(label_ids)
  predictions_probs.append(probabilities)

print('    DONE.')

Predicting labels for 2,737 test sentences...
    DONE.


In [ ]:
flat_predictions = np.concatenate(predictions, axis=0)
flat_predictions_probs = np.concatenate(predictions_probs, axis=0)

flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = np.concatenate(true_labels, axis=0)
avg_test_accuracy = np.sum(flat_predictions == flat_true_labels) / len(flat_true_labels)

print("  Accuracy: {0:.2f}".format(avg_test_accuracy))
print(flat_predictions[10],flat_true_labels[10],flat_predictions_probs[10])

  Accuracy: 0.53
1 1 [0.47487405 0.525126  ]


In [ ]:
df_output = pd.DataFrame(list(zip(list(flat_predictions), list(flat_true_labels),list(flat_predictions_probs))),columns = ['predictions','true_labels','pred_probs'])

In [ ]:
df_test_output = pd.merge(df,df_output,left_index=True,right_index=True,how='inner')

In [ ]:
df_test_output.head()

,stock_date,movement_percent,stock_symbol,market_binary,tweet,predictions,true_labels,pred_probs
0,2015-10-01,-6.53e-03,AAPL,0,rt URL rt stockaced : bzq proshares ultrashort...,1,0,"[0.47411868, 0.52588135]"
1,2015-10-01,-7.36e-03,ABB,0,rt AT_USER icymi : $ abb energizes first phase...,0,0,"[0.5076394, 0.49236062]"
2,2015-10-01,1.32e-02,ABBV,1,rt AT_USER kris jenners ' top picks are $ abbv...,1,1,"[0.46190318, 0.53809685]"
3,2015-10-01,-1.27e-02,AEP,0,aap advance auto parts inc . yield URL $ aap $...,1,0,"[0.46560422, 0.5343958]"
4,2015-10-01,1.38e-02,AMGN,1,amgen $ amgn showing bearish technicals with s...,1,1,"[0.49386203, 0.50613797]"


In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(flat_true_labels, flat_predictions)

0.03448687278075399

In [ ]:
import os
output_dir = './model_save/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)